<a href="https://colab.research.google.com/github/d-atallah/implicit_gender_bias/blob/main/jaymefis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Packages

In [1]:
import gensim.downloader as api
from gensim.matutils import cossim
from gensim.models import KeyedVectors
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

## Load Files

**Annotations** contains crowdsourced annotations for response sentiment and relevance on source-response pairs obtained as described in the paper *RtGender: A Corpus for Studying Differential Responses to Gender* by Rob Voigt, David Jurgens, Vinodkumar Prabhakaran, Dan Jurafsky and Yulia Tsvetkov. Documentation is available [here](https://nlp.stanford.edu/robvoigt/rtgender/).

In [10]:
file_path_annotations = '/content/drive/MyDrive/SIADS 696: Milestone II/Project/Data/RtGender/annotations.csv'
file_path_googlenews = '/content/drive/MyDrive/SIADS 696: Milestone II/Project/Data/RtGender/word2vec-google-news-300.model'

In [53]:
dataframe_annotations = pd.read_csv(file_path_annotations)
dataframe_annotations.head()

,source,op_gender,post_text,response_text,sentiment,relevance
0,facebook_wiki,W,Stopped by Fashion Week and got to hang with A...,You are Both Sweet Ashley Tisdale and Lauren C...,Positive,Poster
1,facebook_wiki,M,"Well guys, real progress is happening. I'm 50 ...",Give us the first page to read. ONE PAGE.,Mixed,Content
2,facebook_wiki,W,Tonight is going to be a good night #PerfectMo...,this is my city was there 2 weeks a go,Neutral,Content
3,facebook_wiki,M,I know grandma Gilmore is real proud of you ht...,if grizzly Adams had a beard.,Neutral,Content
4,facebook_wiki,W,#NEWS to KNOW this AM - Mayor Emanuel will mak...,"Good morning Lourdes, have a great day! Great ...",Positive,Irrelevant


**Google News** contains a pre-trained Word2Vec model based on the Google News dataset, covering approximately 3 million words and phrases. Documentation is available [here](https://code.google.com/archive/p/word2vec/) and [here](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py).

In [8]:
# model_googlenews = api.load('word2vec-google-news-300')
# model_googlenews.save(file_path_googlenews)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [11]:
model_googlenews.save(file_path_googlenews)

In [12]:
model_googlenews = KeyedVectors.load(file_path_googlenews, mmap='r')

In [13]:
for index, word in enumerate(model_googlenews.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(model_googlenews.index_to_key)} is {word}")

word #0/3000000 is </s>
word #1/3000000 is in
word #2/3000000 is for
word #3/3000000 is that
word #4/3000000 is is
word #5/3000000 is on
word #6/3000000 is ##
word #7/3000000 is The
word #8/3000000 is with
word #9/3000000 is said


In [14]:
model_googlenews.most_similar('she')

[('her', 0.7834683060646057),
 ('She', 0.7553189396858215),
 ('herself', 0.669890820980072),
 ('hers', 0.6509943008422852),
 ('he', 0.612994909286499),
 ('woman', 0.5641393661499023),
 ('Rivadineira', 0.558593213558197),
 ('Jana_Bobek', 0.5365386605262756),
 ('mother', 0.5349627137184143),
 ("she'sa", 0.5337814688682556)]

In [15]:
model_googlenews.most_similar('he')

[('He', 0.6712614297866821),
 ('him', 0.6681135892868042),
 ('his', 0.6201768517494202),
 ('she', 0.612994909286499),
 ('himself', 0.588027834892273),
 ('nobody', 0.5637064576148987),
 ('I', 0.555713951587677),
 ('it', 0.5354882478713989),
 ('never', 0.5239652395248413),
 ('somebody', 0.5205153822898865)]

Nouns from v1.1 of the **HolisticBias** dataset, a project of the Responsible Natural Language Processing team at Facebook Research. The dataset is described in the paper **"I'm sorry to hear that": Finding New Biases in Language Models with a Holistic Descriptor Dataset** by Eric Michael Smith, Melissa Hall, Melanie Kambadur, Eleonora Presani, and Adina Williams. Documentation is available [here](https://github.com/facebookresearch/ResponsibleNLP/tree/main/holistic_bias/dataset/v1.1).

In [16]:
nouns = {
    "female": [
        ["woman", "women"],
        ["lady", "ladies"],
        ["gal", "gals"],
        ["girl", "girls"],
        ["mother", "mothers"],
        ["mom", "moms"],
        ["daughter", "daughters"],
        ["wife", "wives"],
        ["grandmother", "grandmothers"],
        ["grandma", "grandmas"],
        ["sister", "sisters"],
        ["sista", "sistas"]
    ],
    "male": [
        ["man", "men"],
        ["bro", "bros"],
        ["guy", "guys"],
        ["boy", "boys"],
        ["father", "fathers"],
        ["dad", "dads"],
        ["son", "sons"],
        ["husband", "husbands"],
        ["grandfather", "grandfathers"],
        ["grandpa", "grandpas"],
        ["brother", "brothers"]
    ],
    "neutral": [
        ["individual", "individuals"],
        ["person", "people"],
        ["kid", "kids"],
        ["parent", "parents"],
        ["child", "children"],
        ["spouse", "spouses"],
        ["grandparent", "grandparents"],
        ["sibling", "siblings"],
        ["veteran", "veterans"]
    ]
}

In [17]:
data_holisticbias = [[gender, ' '.join([word for pair in words_list for word in pair])] for gender, words_list in nouns.items()]
dataframe_holisticbias = pd.DataFrame(data_holisticbias, columns=['gender', 'text'])
dataframe_holisticbias

,gender,text
0,female,woman women lady ladies gal gals girl girls mo...
1,male,man men bro bros guy guys boy boys father fath...
2,neutral,individual individuals person people kid kids ...


## Tokenize Text

In [18]:
stop_words = text.ENGLISH_STOP_WORDS
print(sorted(stop_words))

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both', 'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de', 'describe', 'detail', 'do', 'done', 'down', 'due', 'during', 'each', 'eg', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get', 'give

Several of the stop words in scikit-learn encode gender, including he, him, his, and himself (male); she, her, hers, herself (female); and potentially they, them, their, and themselves (nonbinary). It might be preferable to retain stop words and set a maximum document frequency.

In [19]:
def tokenize_text(dataframe, label_column, text_column, vectorizer):
    """
    Tokenizes text data in a pandas DataFrame using a specified vectorizer.

    Args:
    - dataframe (pd.DataFrame): The DataFrame containing the text data.
    - label_column (str): The name of the column containing the labels.
    - text_column (str): The name of the column containing the text to be tokenized.
    - vectorizer (Vectorizer): An instance of a text vectorization class from sklearn.

    Returns:
    - matrix (sparse matrix): The transformed text data as a matrix.
    - labels (pd.Series): The labels associated with the text data.
    - features (np.ndarray): The features associated with the text data.
    """
    # Remove rows with missing values in the text column
    dataframe = dataframe.dropna(subset=[text_column])

    # Initialize and configure the vectorizer
    vectorizer = vectorizer(
        strip_accents='unicode',
        lowercase=False,
        stop_words=None,
        ngram_range=(1, 1),
        min_df=0.01,
        max_df=0.99,
        max_features=None
    )

    # Transform the text column into a matrix
    matrix = vectorizer.fit_transform(dataframe[text_column])

    # Extract the target variable
    labels = dataframe[label_column]

    # Extract feature names
    features = vectorizer.get_feature_names_out()

    return matrix, labels, features

In [20]:
matrix_count, labels_count, features_count = tokenize_text(dataframe_annotations, 'op_gender', 'response_text', CountVectorizer)
matrix_tfidf, labels_tfidf, features_tfidf = tokenize_text(dataframe_annotations, 'op_gender', 'response_text', TfidfVectorizer)

In [21]:
gender_matrix_count, gender_labels_count, gender_features_count = tokenize_text(dataframe_holisticbias, 'gender', 'text', CountVectorizer)
gender_matrix_tfidf, gender_labels_tfidf, gender_features_tfidf = tokenize_text(dataframe_holisticbias, 'gender', 'text', TfidfVectorizer)

## Apply Word Embeddings

In [22]:
def get_word2vec_vector(matrix, features, model, dimensions=300):
    """
    Generate normalized Word2Vec vectors for documents in a given matrix.

    Args:
    matrix (sparse matrix): A matrix where each row represents a document and each column represents a feature.
    features (np.ndarray): An array of words corresponding to the columns in the matrix.
    model (Word2Vec KeyedVectors): Pre-trained Word2Vec model.
    dimensions (int): Dimensionality of the Word2Vec vectors.

    Yields:
    np.ndarray: Normalized Word2Vec vector for each document.
    """
    # Get the number of documents in the matrix
    num_documents = matrix.shape[0]

    # Iterate over each document in the matrix
    for i in range(num_documents):
        # Initialize the vector for the current document
        document_vector = np.zeros(dimensions)
        # Initialize the total weight for the current document
        document_weight = 0

        # Iterate over each word in the features list
        for j, word in enumerate(features):
            # Check if the word is in the model
            if word in model:
                # Get the weight of the word in the current document
                word_weight = matrix[i, j]
                # Get the vector representation of the word and scale it by its weight
                word_vector = model[word] * word_weight
                # Add the weighted word vector to the document vector
                document_vector += word_vector
                # Accumulate the total weight of the words in the document
                document_weight += word_weight

        # Avoid division by zero
        document_weight = document_weight if document_weight > 0 else 1

        # Yield the normalized document vector
        yield i, document_vector / document_weight

## Calculate Bias

In [51]:
def calculate_bias(female_vector, male_vector, document_vector):
    """
    Calculate the bias in document vectors towards male or female vectors.

    This function computes the cosine similarity between a document vector and
    both female and male vectors. It returns the difference in similarities,
    which can be interpreted as a bias measure towards one of the genders.

    Parameters:
    - female_vector (numpy.ndarray): A vector representing female attributes.
    - male_vector (numpy.ndarray): A vector representing male attributes.
    - document_vector (numpy.ndarray): A vector representing a document.

    Returns:
    - numpy.float64: The difference in cosine similarity between the document vector
      and the female and male vectors, indicating gender bias.
    """

    # Reshape vectors to 2D arrays to fit the input requirement of cosine_similarity
    female_vector = female_vector.reshape(1, -1)
    male_vector = male_vector.reshape(1, -1)
    document_vector = document_vector.reshape(1, -1)

    # Calculate cosine similarity between document and gender vectors
    female_document_similarity = cosine_similarity(female_vector, document_vector)
    male_document_similarity = cosine_similarity(male_vector, document_vector)

    # Return the difference in similarities as a measure of bias
    return float(female_document_similarity - male_document_similarity)

In [54]:
document_word2vec = get_word2vec_vector(matrix_count, features_count, model_googlenews)
_, document_a = next(document_word2vec)
_, document_b = next(document_word2vec)
_, document_c = next(document_word2vec)
_, document_d = next(document_word2vec)
_, document_e = next(document_word2vec)

gender_word2vec = get_word2vec_vector(gender_matrix_count, gender_features_count, model_googlenews)
_, female_vector = next(gender_word2vec)
_, male_vector = next(gender_word2vec)

print(calculate_bias(female_vector, male_vector, document_a))
print(calculate_bias(female_vector, male_vector, document_b))
print(calculate_bias(female_vector, male_vector, document_c))
print(calculate_bias(female_vector, male_vector, document_d))
print(calculate_bias(female_vector, male_vector, document_e))

0.0029751859608960485
-0.0681305181232365
-0.03523238979688181
-0.07459810544660508
-0.06055485557611273


## References

"Please annotate the following code and convert it into PEP 8." OpenAI. (2023). ChatGPT (Jan 30 version) [Large language model]. https://chat.openai.com/chat